In [13]:
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from langchain_tavily import TavilySearch
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent
from langchain_core.messages import HumanMessage

In [11]:
load_dotenv("./.env")

True

In [12]:
memory = MemorySaver()
model = ChatOpenAI(model="gpt-4o-mini", temperature=0.5)
search = TavilySearch(max_results=2)
tools = [search]
agent_executor = create_react_agent(model, tools, checkpointer=memory)

In [15]:
# Use the agent
config = {"configurable": {"thread_id": "abc123"}}

query= "Hi, I'm Bob and I life in SF."
input_message = [HumanMessage(query)]
for step in agent_executor.stream(
    {"messages": input_message}, config, stream_mode="values"
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Hi, I'm Bob and I life in SF.
================================== Ai Message ==================================

Hi Bob! It's great to meet you. How can I assist you today?


In [16]:
input_message = {
    "role": "user",
    "content": "What's the weather where I live?",
}

for step in agent_executor.stream(
    {"messages": [input_message]}, config, stream_mode="values"
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

What's the weather where I live?
================================== Ai Message ==================================
Tool Calls:
  tavily_search (call_TKT7cUyN30qGvuZLJVtVQX4n)
 Call ID: call_TKT7cUyN30qGvuZLJVtVQX4n
  Args:
    query: San Francisco weather
    topic: general
================================= Tool Message =================================
Name: tavily_search

{"query": "San Francisco weather", "follow_up_questions": null, "answer": null, "images": [], "results": [{"title": "Weather in San Francisco, California", "url": "https://www.weatherapi.com/", "content": "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.775, 'lon': -122.4183, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1751754380, 'localtime': '2025-07-05 15:26'}, 'current': {'last_updated_epoch': 1751753700, 'last_updated': '2025-07-05 15:15', 'temp_c': 15.0, 'temp_

# lets check everypart this is like the begining

In [17]:
search = TavilySearch(max_results=2)
search_results = search.invoke("What is the weather in SF")
print(search_results)
# If we want, we can create other tools.
# Once we have all the tools we want, we can put them in a list that we will reference later.
tools = [search]

{'query': 'What is the weather in SF', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'title': 'Weather in San Francisco, CA', 'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.775, 'lon': -122.4183, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1751756289, 'localtime': '2025-07-05 15:58'}, 'current': {'last_updated_epoch': 1751755500, 'last_updated': '2025-07-05 15:45', 'temp_c': 15.3, 'temp_f': 59.5, 'is_day': 1, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/day/116.png', 'code': 1003}, 'wind_mph': 12.1, 'wind_kph': 19.4, 'wind_degree': 259, 'wind_dir': 'W', 'pressure_mb': 1021.0, 'pressure_in': 30.15, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 88, 'cloud': 50, 'feelslike_c': 15.3, 'feelslike_f': 59.5, 'windchill_c': 12.8, 'windchill_f': 55.1, 'heatindex_c': 13.9, 'heatindex_f': 57.0, 'dewpoint_c': 11.9

In [18]:
query = "Hi!"
response = model.invoke([{"role": "user", "content": query}])
response.text()

'Hello! How can I assist you today?'

In [19]:
# create or bind tools to a model
model_with_tools = model.bind_tools(tools)

In [20]:
query = "Hi!"
response = model_with_tools.invoke([{"role": "user", "content": query}])

print(f"Message content: {response.text()}\n")
print(f"Tool calls: {response.tool_calls}")

Message content: Hello! How can I assist you today?

Tool calls: []


In [21]:
query = "Search for the weather in SF"
response = model_with_tools.invoke([{"role": "user", "content": query}])

print(f"Message content: {response.text()}\n")
print(f"Tool calls: {response.tool_calls}")

Message content: 

Tool calls: [{'name': 'tavily_search', 'args': {'query': 'weather in San Francisco', 'topic': 'general'}, 'id': 'call_16McEMfGOKAVpTSEFRmhGw9l', 'type': 'tool_call'}]
